
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 



In [ ]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [1]:
# run study

# add root redis_retrieval_optimizer to path until available on pypi
import sys
import os

# Get the current notebook directory
current_dir = os.path.dirname(os.path.abspath(''))

# Go up two directory levels (adjust the number as needed)
parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add the parent directory to the Python path if it's not already there
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f"Added {parent_dir} to Python path")


import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

/Users/robert.shelton/Library/Caches/pypoetry/virtualenvs/redis-retrieval-optimizer-OYoDaORV-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-13 15:46:11,327] A new study created in memory with name: test


15:46:12 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
15:46:12 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.72it/s]


Recreating index...


Batches: 100%|██████████| 1/1 [00:00<00:00, 10.25it/s]


...

15:47:15 redisvl.index.index INFO   Index already exists, overwriting.
15:47:15 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
15:47:15 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00, 64.35it/s]


Recreating index...
15:47:16 root INFO   Corpus size: 3633
15:47:22 root INFO   Data indexed total_indexing_time=5.348s, num_docs=3633
15:47:23 root INFO   Saving metrics for study: 4d500114-5e56-4b71-a60b-a71c5174f87f, METRICS={'search_method': ['lin_combo', 'lin_combo', 'lin_combo', 'vector', 'vector', 'vector', 'vector', 'vector', 'lin_combo', 'lin_combo'], 'total_indexing_time': [1.959, 3.184, 1.989, 1.761, 3.895, 1.745, 4.108, 2.481, 1.93, 5.348], 'avg_query_time': [0.003249300522700921, 0.0030338048196798507, 0.0032537913543890136, 0.0012082736189520395, 0.0012043253187055559, 0.0013466804020175994, 0.0012582271091709197, 0.001436730287392442, 0.002913703121267974, 0.004260091220631319], 'model': ['sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/all-MiniLM-L

[I 2025-05-13 15:47:24,001] Trial 9 finished with value: 5.348 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'lin_combo', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 10, 'ef_runtime': 50, 'ef_construction': 250, 'm': 64}. Best is trial 9 with value: 5.348.


Completed Bayesian optimization... 


Best Configuration: 9: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'lin_combo', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 10, 'ef_runtime': 50, 'ef_construction': 250, 'm': 64}:


Best Score: [5.348]




In [2]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "recall@k", "precision", "ndcg@k"]].sort_values(by="ndcg@k", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,recall@k,precision,ndcg@k
1,lin_combo,hnsw,float32,250,20,64,0.003034,0.154988,0.243344,0.204229
2,lin_combo,hnsw,float32,100,20,16,0.003254,0.154988,0.243344,0.204229
8,lin_combo,hnsw,float32,150,50,8,0.002914,0.154988,0.243344,0.203669
0,lin_combo,hnsw,float32,100,20,16,0.003249,0.154988,0.243344,0.203651
9,lin_combo,hnsw,float16,250,50,64,0.004260,0.154988,0.243344,0.203651
7,vector,hnsw,float32,150,30,64,0.001437,0.148280,0.241796,0.188781
5,vector,hnsw,float32,100,20,8,0.001347,0.139291,0.230960,0.179333
6,vector,hnsw,float16,300,10,8,0.001258,0.125178,0.224458,0.165261
3,vector,hnsw,float32,100,10,8,0.001208,0.125933,0.226316,0.165205
4,vector,hnsw,float16,150,10,16,0.001204,0.123967,0.225697,0.161683
